# Run clip-retrieval back with fondant-ai/datacomp-small-clip index

### Create virtual environment

In [ ]:
!python3 -m venv .env
!source .env/bin/activate

## Download index and metadata

### Install requirements

In [ ]:
!pip install dask[dataframe] huggingface_hub

### Create the index folder

In [ ]:
!mkdir datacomp_small

### Download the index

In [ ]:
!wget -O datacomp_small/image.index https://huggingface.co/datasets/fondant-ai/datacomp-small-clip/resolve/main/faiss?download=true -q --show-progress

### Download the metadata

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

ddf = dd.read_parquet("hf://datasets/fondant-ai/datacomp-small-clip/id_mapping")
ddf = ddf.rename(columns={"image_path": "url"})
ddf = ddf.repartition(npartitions=1)

with ProgressBar():
    ddf.to_parquet("datacomp_small/metadata")

## Run clip-retrieval backend

### Install requirements

In [ ]:
!pip install clip-retrieval

In [ ]:
%%writefile indices.json
{
    "fondant_datacomp_small": {
        "indice_folder": "datacomp_small",
        "columns_to_return": ["url"],
        "clip_model": "open_clip:ViT-B-32/laion2b_s34b_b79k",
        "enable_mclip_option": false,
        "provide_aesthetic_embeddings": false
    }
}

In [ ]:
!clip-retrieval back --port 1234 --indices-paths indices.json --clip_model open_clip:ViT-B-32/laion2b_s34b_b79k